# **Sat4Ec**: Satellite data for economic flash estimation

`sat4ec` is an algorithm for monitoring the production levels of automotive production plants in Germany using Sentinel-1 IW GRD data. Since cars parked in parking lots of automotive plants interact with the Sentinel-1 SAR wave, a strong or weak signal can be used to infer their presence or absence, respectively. The algorithm aggregates a single mean SAR backscatter value for each AOI and calculates a time series to identify times of high or low parking lot occupancy.

<img src="docs/readme_images/aoi_example.png" width="300">

Individual or all of the following automotive plants can be selected:  <br>
<img src="docs/readme_images/map_facilities.png" width="300">
- *BMW plants* in Regensburg and Leipzig 
- *Audi plants* in Ingolstadt and Neckarsulm
- *Ford plants* in Cologne and Saarlouis
- *Mercedes plants* in Bremen, Düsseldorf and Ludwigsfelde
- *Opel plants* in Eisenach and Rüsselsheim
- *Porsche plant* in Leipzig
- *VW plants* in Emden, Wolfsburg and Zwickau 

For each plant, production parking lots can be analyzed individually or collectively on the basis of daily or monthly data. 

**Example Output for BMW Regensburg**

<img src="docs/readme_images/s1_pixels.png" width="250">
<img src="docs/readme_images/s1_pixels_mean.png" width="250">
<img src="docs/readme_images/bmw.png" width="455">

**Prerequisites:**
This notebook requires an active [Sentinel Hub](https://www.sentinel-hub.com/) subscription.

#### Running the algorithm

In [1]:
from sat4ec.execution.exe_production import Production
from pathlib import Path
import os
from IPython.display import display, Image

In [2]:
# Capture credentials from the user
sh_client_id = input("Enter your Sentinel Hub Client ID: ")
sh_client_secret = input("Enter your Sentinel Hub Secret Client ID: ")

# Validate credentials
if not sh_client_id or not sh_client_secret:
    raise ValueError("No valid Sentinel Hub credentials available. Please provide valid credentials.")

# Save the credentials as environment variables
os.environ["SH_CLIENT_ID"] = sh_client_id
os.environ["SH_CLIENT_SECRET"] = sh_client_secret

In [3]:
# choose one or more automotive facilties to analyze by commenting out all others

aois = [
    "bmw_regensburg",
    "bmw_leipzig",
    # "audi_ingolstadt",
    # "audi_neckarsulm",
    # "ford_cologne",
    # "ford_saarlouis",
    # "mercedes_bremen",
    # "mercedes_duesseldorf",
    # "mercedes_ludwigsfelde",
    # "opel_eisenach",
    # "opel_ruesselsheim",
    # "porsche_leipzig",
    # "vw_emden",
    # "vw_wolfsburg",
    # "vw_zwickau",
]

In [ ]:
from sat4ec.execution.exe_config import Config

for aoi in aois:
    print(f"Preparing data for automotive facility {aoi}.")

    conf = Config(
        aoi_dir=Path(r"docs\aois"),
        working_dir=Path(r"sat4ec"),
        out_dir=Path(r"output"),
        orbit="des",  # choose between ascending or descending orbit or both
        aoi=aoi,
        ext="geojson",
        start=None,  # specify a starting date for the time series (YYYY-MM-DD) or comment this line or enter None if using default start date
        end=None,  # specify an end date for the time series (YYYY-MM-DD) or comment this line or enter None if using automatic end date
        monthly=True, # choose between monthly (True) and daily (False) data
        regression="spline", # choose between available interpolation methods for daily data: spline (default), rolling mean, polynomial
        linear=True,
        linear_fill=False,
        aoi_split=False, # choose between data for individual parking lots (True) or entire facilities (False)
        overwrite_raw=False,
        online=True,
        pol="VH",
    )
    
    prod = Production(config=conf)
    prod.workflow()

#### Displaying the plots

In [ ]:
def find_png_files(aoi):
    plot_location = os.path.join("output", aoi, "plot", "png")
    png_files = []
    if os.path.isdir(plot_location):
        for file in os.listdir(plot_location):
            if file.endswith(".png"):
                png_files.append(os.path.join(plot_location, file))
    return png_files

for aoi in aois:
    print("Plots for automotive facility", aoi)
    png_files = find_png_files(aoi)
    for png_file in png_files:
        display(Image(filename=png_file, width=600))